In [2]:
import pandas as pd
from pathlib import Path
import plotly.express as px
from utils import *

In [3]:
START_DATE = '2023-03-01'
END_DATE = '2023-07-02'

interactions_file = Path('../..') / 'data' / 'my_intervention' / 'interactions_per_day_rjba.csv'


In [4]:
interactions = pd.read_csv(interactions_file)
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    552 non-null    object
 1   State   552 non-null    object
 2   total   552 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 13.1+ KB


In [5]:
interactions = interactions[(interactions['Date'] >= START_DATE) & (interactions['Date'] <= END_DATE)]

interactions.shape


(372, 3)

In [6]:
interactions_per_day_rj = interactions[interactions['State'] == 'Rio de Janeiro'].copy()

interactions_per_day_rj.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124 entries, 178 to 547
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    124 non-null    object
 1   State   124 non-null    object
 2   total   124 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.9+ KB


# Plot

In [7]:

# plot cumsum
fig = px.bar(interactions_per_day_rj, x='Date', y='total', title="", template='presentation')

fig.update_layout(template='presentation',
                  # rename total_sum to 'Total replies'
                  showlegend=False,
                  xaxis_title='',
                  yaxis_title="Interactions with gun violence reports")

fig.add_vline(x=INTERVENTION_DATE, line_width=3, line_dash="dash", line_color="green")

# add a text
fig.add_annotation(
    x=INTERVENTION_DATE, y=0,
    text="Intervention started",
    showarrow=False,
    yshift=650,
    xshift=80
)

fig.update_xaxes(showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True)

fig.update_yaxes(showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True)


fig.show()

In [8]:
# # export img
# fig.write_image("../../../thesis_sds_latex/figs/interaction_after.png", width=1200, height=800, scale=2)

fig.write_image(img_folder / "interaction_after.png", width=1200, height=800, scale=2)

# Trendline

In [17]:
interactions_per_day = interactions[interactions.State != 'Pernambuco'].copy()

interactions_per_day.groupby(['State', 'Date']).size().reset_index(name='total')

# convert Date to datetime
interactions_per_day['Date'] = pd.to_datetime(interactions_per_day['Date'])


In [18]:
fig = px.scatter(interactions_per_day, y='total', x='Date', color='State', facet_col_wrap=3, template='presentation', trendline='ols',
             title='',     
        
             labels={'total': 'Interactions with gun violence reports', 'Date': 'Date'})

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period",
)

fig.update_layout(
    # show x-axis as date but only month
    xaxis=dict(
        tickformat='%b',
        tickmode='array',
        tickvals=['2023-03-01', '2023-04-01', '2023-05-01'],
        ticktext=['Mar 2023', 'Apr', 'May',],
        title='',
    ),
)


fig.update_xaxes(showline=True,
         linewidth=1,
         linecolor='black',
         mirror=True)

fig.update_yaxes(showline=True,
         linewidth=1,
         linecolor='black',
         mirror=True)

# disable zeroline
fig.update_yaxes(zeroline=False)